# A2L

> A2L parsing and writing library

[![Build Status](https://travis-ci.org/mborn319/a2l.svg?branch=master)](https://travis-ci.org/mborn319/a2l)
[![Coverage Status](https://coveralls.io/repos/github/mborn319/a2l/badge.svg?branch=master)](https://coveralls.io/github/mborn319/a2l?branch=master)
[![PyPI version](https://badge.fury.io/py/a2l.svg)](https://badge.fury.io/py/a2l)
[![Documentation Status](https://readthedocs.org/projects/a2l/badge/?version=latest)](https://a2l.readthedocs.io/en/latest/?badge=latest)

## Installation

1. Install pya2l 

from PyPI (recommended): 
```python
pip install pya2l
```

or from the forked repository with extra resource a2l example files:
```bash
pip install git+https://github.com/binjian/pya2l.git@master
```

2. Install a2ltool (need rust and cargo tool)
by cloning 
```bash
git clone https://github.com/DanielT/a2ltool.git
```
and then run
```bash
cargo build --release
```

3. Fix the faulty VBU.a2l file and get the VBU.json file
	- open the VBU.a2l file with a text editor
	- Find 4 0x lines by running a2ltool and change them to 0xFFFFFFFF 
		```bash
	 	./a2ltool VBU.a2l -a 1.6.1    
	 	```
	- Find lines with missing included a2l file (EEPROM.a2l) by running a2ltool like previously and remove
	- Fine  faulty lines with DISPLAY_IDENTIFIER, RAM, RAM_INIT_BY_ECU and remove by running pya2l
		```bash
 		pya2l -v res/VBU.a2l to_json -o res/VBU.json -i 2 
		```
	- When no error is reported, the VBU.json file is ready to use

In [ ]:
#| default_exp a2l

In [ ]:
#| hide
from __future__ import annotations
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| hide
from IPython.core.interactiveshell import InteractiveShell

In [ ]:
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
#| export
import ijson
import json
import inspect
from typing import Optional, Union, List
from functools import cached_property, cache
import re
from enum import Enum
from pathlib import Path
from pprint import pprint
import argparse
from InquirerPy import inquirer
from InquirerPy.validator import NumberValidator, EmptyInputValidator, PathValidator
from InquirerPy.base.control import Choice
from pydantic import BaseModel, Field, validator, conlist
from typing_extensions import Annotated

In [ ]:
#| export
def list_of_strings(strings: str)->list[str]:
	""" split a string separated by ',', ';', or '\s' to a list of strings.
	Descripttion: split a long string to a list of strings.

	Args:
		strings (str): The string to split.

	Returns:
		list: The list of strings.
	"""
	return re.split(r',\s*|;\s*|\s+', strings)	


In [ ]:
test_eq(list_of_strings(r'foo;123, bar ebi'), ['foo','123','bar','ebi'])
list_of_strings(r'"/PROJECT/MODULE[0]/MOD_COMMON, /PROJECT/IF_DATA[0]/Blob[0]/, /PROJECT/MODULE[0]/CHARACTERISTIC, TQD_trqTrqSetNormal_MAP_v"')

['"/PROJECT/MODULE[0]/MOD_COMMON',
 '/PROJECT/IF_DATA[0]/Blob[0]/',
 '/PROJECT/MODULE[0]/CHARACTERISTIC',
 'TQD_trqTrqSetNormal_MAP_v"']

In [ ]:
#| export
class JsonNodePathSegment:
	"""result of parsing json node path segment
	
	Args:
		name (str): name of the node
		indices (list[int]): indices of the node
		index_range (list[int]): index range of the node

		if both indices and index_range are None, then the node is a dict, otherwise it is a list

	"""
	def __init__(self, name: str, indices: list[int]=None, index_range: list[int]=None):
		self.name = name
		self.indices = indices
		self.index_range = index_range
		assert (
			(indices is None and index_range is None)
			or (indices is [] and index_range is None)  # empty list is valid for lazy loading
			or (indices is not None and index_range is None) 
			or (indices is None and index_range is not None)
		), 'Invalid JsonNodePathSegment'

	def __repr__(self):
		if self.indices is None and self.index_range is None:
			return f'<{self.name} dict>'
		elif self.indices == []:
			return f'<{self.name}[] list>'
		elif self.indices is not None:
			return f'<{self.name}[{",".join([str(i) for i in self.indices])}] list>'
		elif len(self.index_range)==2:  #self.index_range is not None:
			return f'<{self.name}[{self.index_range[0]}:{self.index_range[1]}] list>'
		elif len(self.index_range)==3:  #self.index_range is not None:
			return f'<{self.name}[{self.index_range[0]}:{self.index_range[1]}:{self.index_range[2]}] list>'
		else:
			raise ValueError(f'Invalid index range {self.index_range}')

	@property	
	def is_dict(self):
		return self.indices is None and self.index_range is None

In [ ]:
#| export
class JsonNodePath:
	"""result of parsing json node path
	
	Args:
		segments (list[JsonNodePathSegment]): list of JsonNodePathSegment

	"""
	def __init__(self, node_path: str):
		""" Parse the json data to get the node specified by the node path.
		Descripttion: Parse the json data to get the node specified by the node path.

		Args:
			node_path (str): The node path to the node.
			json_data (dict): The json data to parse.

		Returns:
			dict: The node specified by the node path.
		"""
		self.node_path_str = node_path
		path_segments = re.split(r'/\s*', node_path)[1:]
		self.node_path_segments = []
		for s in path_segments:
			name = re.search(r'(\w+)', s).group(1)
			# res = re.search('(?:\[(\d+)((?:(,?\s*(\d*))*)|(?:(:?\s*(\d*))*))\])', s)  # regex with \d+ for mandatory digit in [] pair
			res = re.search('(?:\[(\d*)((?:(,?\s*(\d*))*)|(?:(:?\s*(\d*))*))\])', s)  # regex with \d* for optional empyt [] pair
			if res:
				r = res.groups()
				if r[0]=='' and r[1]=='':
					# print(f'{name} is a list : index: {r[0]}')
					self.node_path_segments.append(JsonNodePathSegment(name=name,indices=[]))  # append empty list for lazy finding and loading
				elif r[0]!='' and r[1]=='':
					# print(f'{name} is a list : index: {r[0]}')
					self.node_path_segments.append(JsonNodePathSegment(name=name,indices=[int(r[0])]))
				elif ',' in r[1]:
					indices = re.split(r',\s*', r[1])
					indices[0] = r[0]
					# print(f'{name} is a list : indices {indices}')
					self.node_path_segments.append(JsonNodePathSegment(name=name,indices=indices))
				elif ':' in r[1]:
					index_range = re.split(r':\s*', r[1])
					index_range[0] = r[0]
					# if len(index_range)==2:
					# 	print(f'{name} is a list : index range [{index_range[0]}:{index_range[1]}]')
					# elif len(index_range)==3:
					# 	print(f'{name} is a list : index range [{index_range[0]}:{index_range[1]}:{index_range[2]}]')
					# else:
					# 	raise ValueError(f'Invalid index range {index_range}')
					self.node_path_segments.append(JsonNodePathSegment(name=name,index_range=index_range))
				else:
					raise ValueError(f'Invalid index spec in {s}')
			else:
				# print(f'{name} is a dict')
				self.node_path_segments.append(JsonNodePathSegment(name=name))
	def __repr__(self):
		return f'<JsonNodePath {self.node_path_segments}>'

	def __iter__(self):
		return (s for s in self.node_path_segments)	

	@property
	def leaf(self):
		"""return the leaf of the node path"""
		return self.node_path_segments[-1]

	@property
	def lazy_path(self):
		"""return the lazy path of the node path"""
		return '.'.join([f'{s.name}' if s.is_dict else f'{s.name}.item' for s in self.node_path_segments])

In [ ]:
node_path = r"/PROJECT[0]/MODULE[0,3,5]/IF_DATA[3:2:8]/CHARACTERISTIC[3:5]/TQD_trqTrqSetNormal_MAP_v"
jnode_path = JsonNodePath(node_path)
print(jnode_path)
res = ['<PROJECT[0] list>', '<MODULE[0,3,5] list>', '<IF_DATA[3:2:8] list>', '<CHARACTERISTIC[3:5] list>', '<TQD_trqTrqSetNormal_MAP_v dict>'] 
for (s,r) in zip(jnode_path,res):
	print(s)
	test_eq(s.__str__(), r)

<JsonNodePath [<PROJECT[0] list>, <MODULE[0,3,5] list>, <IF_DATA[3:2:8] list>, <CHARACTERISTIC[3:5] list>, <TQD_trqTrqSetNormal_MAP_v dict>]>
<PROJECT[0] list>
<MODULE[0,3,5] list>
<IF_DATA[3:2:8] list>
<CHARACTERISTIC[3:5] list>
<TQD_trqTrqSetNormal_MAP_v dict>


In [ ]:
#| export
def get_argparser()->argparse.ArgumentParser:
	""" Get the argument parser for the command line interface.
	Descripttion: Get the argument parser for the command line interface.

	Returns:
		argparse.ArgumentParser: The argument parser for the command line interface.
	"""
	import re

	parser = argparse.ArgumentParser(
        "Get the A2L file path and the desired configuration for CCP/XCP.",
    )

	parser.add_argument(
        "-p",
        "--path",
        type=str,
        default=r"../res/vbu_sample.json",
        help="path to the A2L file",
    )

	parser.add_argument(
		"-n",
		"--node-path",
		type=JsonNodePath,
		default=r"/PROJECT/MODULE[]",
		# default=r"/PROJECT/MODULE[]/CHARACTERISTIC[], "
		# 		r"/PROJECT/MODULE[]/MEASUREMENT[], "
		# 		r"/PROJECT/MODULE[]/AXIS_PTS[], "
		# 		r"/PROJECT/MODULE[]/COMPU_METHOD[], ",
		help="node path to search for calibration parameters",
	)

	parser.add_argument(
		"-l",
		"--leaves",
		type=list_of_strings,
		default=r"TQD_trqTrqSetNormal_MAP_v, " 
				r"VBU_L045A_CWP_05_09T_AImode_CM_single, " 
				r"Lookup2D_FLOAT32_IEEE, " 
				r"Lookup2D_X_FLOAT32_IEEE, " 
				r"Scalar_FLOAT32_IEEE, " 
				r"TQD_vVehSpd, "
				r"TQD_vSgndSpd_MAP_y, "
				r"TQD_pctAccPedPosFlt, "
				r"TQD_pctAccPdl_MAP_x",
			help="leaf nodes to search for",
	)

	return parser

In [ ]:
#| export
parser = get_argparser()
args = parser.parse_args(
	[
		"-p",
		# r"../res/VBU_AI.json",
		r"../res/vbu_sample.json",
		"-n",
		r"/PROJECT/MODULE[], ",
		# r"/PROJECT/MODULE[]/CHARACTERISTIC[], "
		# 	r"/PROJECT/MODULE[]/MEASUREMENT[], "
		# 	r"/PROJECT/MODULE[]/AXIS_PTS[], "
		# 	r"/PROJECT/MODULE[]/COMPU_METHOD[]",
		"-l",
		r"TQD_trqTrqSetNormal_MAP_v, " 
				r"VBU_L045A_CWP_05_09T_AImode_CM_single, " 
				r"Lookup2D_FLOAT32_IEEE, " 
				r"Lookup2D_X_FLOAT32_IEEE, " 
				r"Scalar_FLOAT32_IEEE, " 
				r"TQD_vVehSpd, "
				r"TQD_vSgndSpd_MAP_y, "
				r"TQD_pctAccPedPosFlt, "
				r"TQD_pctAccPdl_MAP_x",
	]
)
args.__dict__

{'path': '../res/vbu_sample.json',
 'node_path': <JsonNodePath [<PROJECT dict>, <MODULE[] list>]>,
 'leaves': ['TQD_trqTrqSetNormal_MAP_v',
  'VBU_L045A_CWP_05_09T_AImode_CM_single',
  'Lookup2D_FLOAT32_IEEE',
  'Lookup2D_X_FLOAT32_IEEE',
  'Scalar_FLOAT32_IEEE',
  'TQD_vVehSpd',
  'TQD_vSgndSpd_MAP_y',
  'TQD_pctAccPedPosFlt',
  'TQD_pctAccPdl_MAP_x']}

In [ ]:
# jnode_paths = []
# for p in args.node_paths:
# 	print(p)
# 	jnode_path = JsonNodePath(p)
# 	print(jnode_path)
# 	jnode_paths.append(jnode_path)
# 	pprint(jnode_path.lazy_path)
# 	pprint(re.split(r'\.', jnode_path.lazy_path)) 
# 	pprint(jnode_path.leaf.name)
# 	print()
print(args.node_path)
# jnode_path = JsonNodePath(args.node_path)
# print(jnode_path)
pprint(args.node_path.lazy_path)
pprint(re.split(r'\.', args.node_path.lazy_path)) 
pprint(args.node_path.leaf.name)
print()
# node_path = r"/PROJECT/MODULE[]/CHARACTERISTIC[]"
# node_path = args.node_path
# jnode_path = JsonNodePath(node_path)
# res = ['<PROJECT[0] list>', '<MODULE[0,3,5] list>', '<IF_DATA[3:2:8] list>', '<CHARACTERISTIC[3:5] list>', '<TQD_trqTrqSetNormal_MAP_v dict>'] 
for s in args.node_path:
	print(s)

<JsonNodePath [<PROJECT dict>, <MODULE[] list>]>
'PROJECT.MODULE.item'
['PROJECT', 'MODULE', 'item']
'MODULE'

<PROJECT dict>
<MODULE[] list>


In [ ]:
node_path = r"/PROJECT/MODULE[]/CHARACTERISTIC[]"
re.split(r',\s*|;\s*|/\s*|\s+', node_path)	
re.split('(?:\[\d\])',  'foo, a[0], bar')

['', 'PROJECT', 'MODULE[]', 'CHARACTERISTIC[]']

['foo, a', ', bar']

In [ ]:
prog = re.compile('(\[\d\])')
result = prog.search('/PROJECT/MODULE[0]/CHARACTERISTIC[0]').group(1)
print(result)

[0]


In [ ]:
re.search('(?:\[(\d)\])',  'foo, a[0], bar').groups()
re.search('(?:\[(\d,?\s*\d*)\])',  'foo, a[0, 4], bar').groups()

('0',)

('0, 4',)

In [ ]:
try:
	res = re.search('(?:\[(\d+),?\s*(\d*)\])',  'foo, a[0, 24], bar').groups()
	print(res)
	# re.search('(?:\[(\d,?\s*\d*)\])', 'foo, a[0, 4], bar').groups()
except AttributeError as exc:
	print(exc)

('0', '24')


In [ ]:
try:
	res = re.search('(?:\[(\d+),?\s*(\d*)\])',  'foo, bar').groups()
	print(res)
	# re.search('(?:\[(\d,?\d*)\])',  'foo').groups()
except AttributeError as exc:
	print(exc)

'NoneType' object has no attribute 'groups'


In [ ]:
# node_path = args.node_paths[0]
test_node_path = r"/PROJECT[0]/MODULE[0,3,5]/IF_DATA[3:2:8]/CHARACTERISTIC[3:5]/TQD_trqTrqSetNormal_MAP_v"
path_segments = re.split(r'/\s*', test_node_path)[1:]
print(path_segments)


['PROJECT[0]', 'MODULE[0,3,5]', 'IF_DATA[3:2:8]', 'CHARACTERISTIC[3:5]', 'TQD_trqTrqSetNormal_MAP_v']


In [ ]:
# prefix = '.'.join(jnode_path.lazy_path.split('.')[:-1])
prefix = args.node_path.lazy_path
args.path, prefix
objects = ijson.items(open(args.path, 'r'), prefix)
module_items = list(objects)[0]
# pprint(module_items)
pprint(module_items['CHARACTERISTIC'][0]['Name'])
print('\n')
l = {k:v for k,v in module_items['CHARACTERISTIC'][0]['AXIS_DESCR'][0].items()}
pprint(l)


('../res/vbu_sample.json', 'PROJECT.MODULE.item')

{'Value': 'TQD_trqTrqSetNormal_MAP_v'}


{'AXIS_PTS_REF': {'AxisPoints': {'Value': 'TQD_vSgndSpd_MAP_y'}},
 'Attribute': 'COM_AXIS',
 'Conversion': {'Value': 'VBU_L045A_CWP_05_09T_AImode_CM_single'},
 'InputQuantity': {'Value': 'TQD_vVehSpd'},
 'LowerLimit': {'DecimalSize': 5,
                'IntegralSign': '-',
                'IntegralSize': 1,
                'Value': Decimal('-3.4E+38')},
 'MaxAxisPoints': {'Base': 10, 'Size': 2, 'Value': 14},
 'UpperLimit': {'DecimalSize': 5,
                'IntegralSize': 1,
                'Value': Decimal('3.4E+38')}}


In [ ]:
args.path, args.node_path, args.leaves

('../res/vbu_sample.json',
 <JsonNodePath [<PROJECT dict>, <MODULE[] list>]>,
 ['TQD_trqTrqSetNormal_MAP_v',
  'VBU_L045A_CWP_05_09T_AImode_CM_single',
  'Lookup2D_FLOAT32_IEEE',
  'Lookup2D_X_FLOAT32_IEEE',
  'Scalar_FLOAT32_IEEE',
  'TQD_vVehSpd',
  'TQD_vSgndSpd_MAP_y',
  'TQD_pctAccPedPosFlt',
  'TQD_pctAccPdl_MAP_x'])

In [ ]:
calibs = []
node_paths = [r"/PROJECT/MODULE[]/CHARACTERISTIC[]",
			r"/PROJECT/MODULE[]/MEASUREMENT[]",
			r"/PROJECT/MODULE[]/AXIS_PTS[]",
			r"/PROJECT/MODULE[]/COMPU_METHOD[]"]
jnode_paths = [JsonNodePath(p) for p in node_paths]

for jp in jnode_paths:
	prefix = jp.lazy_path
	objects = ijson.items(open(args.path, "r"), prefix)
	# calib = [o for o in objects for k, v in o.items() if k == 'Name']
	# pprint(calib)
	# len(calib)
	# print(prefix)
	for o in objects:
		for k, v in o.items():
			if k == 'Name':
				if v['Value'] in args.leaves:
					calibs.append(o)

pprint(calibs[3])
# calib['LowerLimit']['Value'], calib['UpperLimit']['Value']
	
# for c in calibs:
# 	pprint(c)

{'Address': {'Base': 16, 'Size': 8, 'Value': '1879071450'},
 'Conversion': {'Value': 'VBU_L045A_CWP_05_09T_AImode_CM_single'},
 'DepositR': {'Value': 'Lookup2D_X_FLOAT32_IEEE'},
 'InputQuantity': {'Value': 'TQD_vVehSpd'},
 'LongIdentifier': {},
 'LowerLimit': {'DecimalSize': 5,
                'IntegralSign': '-',
                'IntegralSize': 1,
                'Value': Decimal('-3.4E+38')},
 'MaxAxisPoints': {'Base': 10, 'Size': 2, 'Value': 14},
 'MaxDiff': {},
 'Name': {'Value': 'TQD_vSgndSpd_MAP_y'},
 'UpperLimit': {'DecimalSize': 5,
                'IntegralSize': 1,
                'Value': Decimal('3.4E+38')}}


In [ ]:
#| export
class Bunch(object):
	"""collector of a bunch of named stuff into one object; a generic record/struct type, indexed by keys"""
	bunch_registry = {} 
	def __init__(self, key, **kwargs):
		"""Bunch object self contains no 'key' attribute, but adict could have."""
		self.key = key
		self.__dict__.update(kwargs)
		self.__class__.bunch_registry.update({key: self})
	
	def __repr__(self):
		return f'<{self.__class__.__name__}.{self.key}>'
	
	def __hash__(self) -> int:
		return hash(tuple(sorted(self.__dict__.items())))
	
	def __eq__(self, other) -> bool: 
		return self.__dict__ == other.__dict__
	
	@staticmethod
	def fetch(key):
		return Bunch.__index[key]
	
	# @classmethod
	# def register(cls, key: str, value: bunch):
	# 	"""manual registration of a bunch object with a key
		
	# 	args:
	# 		key: jnode_path string
	# 		value: bunch object
			
	# 	"""
	# 	cls.bunch_registry.update({key: value})
	

In [ ]:
#| export
class Record:
	"""object with dynamic attributes"""
	record_registry = None
	__RecordCats = None
	__cat = None 
	subclass_registry = {}

	def __init__(self, **kwargs):
		self.__dict__.update(kwargs)

	def __repr__(self):
		return f'<{self.__class__.__name__}: {self.Name!r}>'

	@staticmethod
	def fetch(key: str) -> Union[Record, str]:
		try:
			rec = Record.record_registry[key]
		except KeyError:
			rec = key.split('.')[-1]

		return rec 
	
	@classmethod
	def load_types(cls, path: Path, jnode_path: Optional[JsonNodePath] = JsonNodePath('/PROJECT/MODULE[]')) -> None:
		"""
		Load types for the Record class.

		Args:
			path (Path): The path to the file.
			jnode_path (Optional[JsonNodePath], optional): The JSON node path. Defaults to JsonNodePath('/PROJECT/MODULE[]').
		"""
		cls.__RecordCats = load_class_type_a2l_lazy(path, jnode_path)

	@classmethod
	def load_records(cls, path: Path, keys: list[str], jnode_path: Optional[JsonNodePath] = JsonNodePath('/PROJECT/MODULE[]')) -> None:
		"""
		Load records for the Record class.

		Args:
			path (Path): The path to the file.
			keys (list[str]): The list of keys.
			jnode_path (Optional[JsonNodePath], optional): The JSON node path. Defaults to JsonNodePath('/PROJECT/MODULE[]').
		"""
		cls.load_types(path, jnode_path)
		cls.record_registry = load_records_lazy(path, keys, jnode_path)

In [ ]:
#| export 
class Calibration(Record):
	"""Target calibration object for torque map; a2l section ["PROJECT"]["MODULE"]["CHARACTERISTIC"]
	
	First level keys will be turned into attributes of the object, encoded registered values will be replaced with the corresponding objects.
	Otherwiese the key-value pairs will be kept as is. 
	""" 
	__cat = 'CHARACTERISTIC'
	Record.subclass_registry[__cat] = 'Calibration'

	def __init__(self, **kwargs):
		# update the dict with the kwargs
		super().__init__(**kwargs)
		# TODO (optionally) add check for the class existence of the keys with ijson parser event handler


	def __repr__(self):
		try:
			return f'<{self.__class__.__name__}: {self.Name!r}>'
		except AttributeError:
			return super().__repr__() 

	@cached_property
	def data_conversion(self):
		try:
			key = self.__dict__['Conversion']['Value']  # define the key for the axis for future fetch
		except KeyError:
			raise KeyError(f'The key "Conversion" is not found in the calibration description or '
						f'the key "Value" is not found in the "Conversion" dict.')
		cat = 'COMPU_METHOD'
		key = f"{super().subclass_registry[cat]}.{key}"  # define the key for the axis for future fetch
		return self.__class__.fetch(key)

	@cached_property
	def record_type(self):
		try:
			key = self.__dict__['Deposit']['Value']  # define the key for the axis for future fetch
		except KeyError:
			raise KeyError(f'The key "Deposit" is not found in the calibration description or '
						f'the key "Value" is not found in the "Deposit" dict.')
		cat = 'RECORD_LAYOUT'
		key = f"{super().subclass_registry[cat]}.{key}"  # define the key for the axis for future fetch
		rtype = self.__class__.fetch(key)
		if type(rtype) is str:  # if the key is not found in the registry, then it is a scalar
			key = 'Scalar_' + rtype  # construct the new key for the scalar as defined in a2l
			cat = 'RECORD_LAYOUT'
			rtype = self.__class__.fetch(key)
		
		return rtype

	@cached_property
	def address(self):
		return hex(int(self.__dict__['Address']['Value']))[2:]   # transform Ecu address to hex string without '0x'

	@cached_property
	def axes(self):  # y axis
		"""_summary_

			for axes[0/1]['InputQuantity']['Value] == 'TQD_vVehSpd' or 'TQD_pctAccPedPosFlt':
		Raises:
			ValueError: _description_
			KeyError: _description_
			KeyError: _description_
			KeyError: _description_

		Returns:
			_type_: _description_
		"""
		try:
			axes = []
			for axis in self.__dict__['AXIS_DESCR']: # 'AXIS_DESCR' is a list of dicts 2 axes for 2D map 
				try:
					if axis['Attribute'] != 'COM_AXIS':
						raise ValueError(f'The value of "Attribute" {axis["Attribute"]} is not "COM_AXIS".')
				except KeyError:
					raise KeyError('The key "Attribute" is not found in the axis description.')
				try:
					key = axis['InputQuantity']['Value']  # define the key for the axis for future fetch
				except KeyError:
					raise KeyError(f'The key "InputQuantity" is not found in the axis description or '
								f'the key "Value" is not found in the "InputQuantity" dict.')
				# get the MEASUREMENT object from the registry				
				cat = 'MEASUREMENT'
				key = f"{super().subclass_registry[cat]}.{key}"
				axis['measurement'] = self.__class__.fetch(key) # replace value with the object

				try:
					key = axis['AXIS_PTS_REF']['AxisPoints']['Value']  # define the key for the axis for future fetch
				except KeyError:
					raise KeyError(f'The key "AXIS_PTS_REF " is not found in the axis description or '
								f'the key "AxisPoins" is not found in the "AXIS_PTS_REF" dict.'
								f'the key "Value" is not found in the "AxisPoints" dict.')
				# get the AXIS_PTS object from the registry				
				cat = 'AXIS_PTS'
				key = f"{super().subclass_registry[cat]}.{key}"
				axis['axis_scale']= self.__class__.fetch(key)  # replace value with object 

				try:
					key = axis['Conversion']['Value']  # define the key for the axis for future fetch
				except KeyError:
					raise KeyError(f'The key "Conversion" is not found in the axis description or '
								f'the key "Value" is not found in the "Conversion" dict.')
				# get the AXIS_PTS object from the registry				
				cat = 'COMPU_METHOD'
				key = f"{super().subclass_registry[cat]}.{key}"
				axis['data_conversion'] = self.__class__.fetch(key)  # replace value with object

				bunch_register_key = f'{self.__class__.__name__}.{self.Name}.{axis["InputQuantity"]["Value"]}' 
				bunch = Bunch(bunch_register_key,**axis)
				axes.append(bunch)
				# Bunch.register(bunch_register_key,bunch)
			
			return axes
		except KeyError:
			raise KeyError('The key "AXIS_DESCR" is not found in the calibration object.')

In [ ]:
#| export
class Measurement(Record):
	"""Measurement object like speed,  acc pedal position, etc; a2l section ["PROJECT"]["MODULE"]["MEAUREMENT"]]""" 
	__CAT = 'MEASUREMENT'
	Record.subclass_registry[__CAT] = 'Measurement'

	def __repr__(self):
		try:
			return f'<{self.__class__.__name__}: {self.Name!r}>'
		except AttributeError:
			return super().__repr__() 
	
	@cached_property
	def data_conversion(self):
		try:
			key = self.__dict__['Conversion']['Value']  # define the key for the axis for future fetch
		except KeyError:
			raise KeyError(f'The key "Conversion" is not found in the calibration description or '
						f'the key "Value" is not found in the "Conversion" dict.')
		cat = 'COMPU_METHOD'
		key = f"{super().subclass_registry[cat]}.{key}"
		return self.__class__.fetch(key)

	@cached_property
	def record_type(self):
		try:
			key = self.__dict__['DataType']['Value']  # define the key for the axis for future fetch
		except KeyError:
			raise KeyError(f'The key "Deposit" is not found in the calibration description or '
						f'the key "Value" is not found in the "Deposit" dict.')
		cat = 'RECORD_LAYOUT'
		key = f"{super().subclass_registry[cat]}.{key}"  # define the key for the axis for future fetch
		rtype = self.__class__.fetch(key)
		if type(rtype) is str:  # if the key is not found in the registry, then it is a scalar
			key = 'Scalar_' + rtype  # construct the new key for the scalar as defined in a2l
			cat = 'RECORD_LAYOUT'
			rtype = self.__class__.fetch(key)
		
		return rtype

	@cached_property
	def address(self):
		return hex(int(self.__dict__['ECU_ADDRESS']['Value']))[2:]   # transform Ecu address to hex string without '0x'

In [ ]:
#| export
class AxisScale(Record):
	"""Target calibration object for torque map; a2l section ["PROJECT"]["MODULE"]["AXIS_PTS"]""" 
	__CAT = 'AXIS_PTS'
	Record.subclass_registry[__CAT] = 'AxisScale'

	def __repr__(self):
		try:
			return f'<{self.__class__.__name__}: {self.Name!r}>'
		except AttributeError:
			return super().__repr__() 
	
	@cached_property
	def data_conversion(self):
		try:
			key = self.__dict__['Conversion']['Value']  # define the key for the axis for future fetch
		except KeyError:
			raise KeyError(f'The key "Conversion" is not found in the calibration description or '
						f'the key "Value" is not found in the "Conversion" dict.')
		cat = 'COMPU_METHOD'
		key = f"{super().subclass_registry[cat]}.{key}"
		return self.__class__.fetch(key)
	
	@cached_property
	def record_type(self):
		try:
			key = self.__dict__['DepositR']['Value']  # define the key for the axis for future fetch
		except KeyError:
			raise KeyError(f'The key "DepositR" is not found in the calibration description or '
						f'the key "Value" is not found in the "DepositR" dict.')
		cat = 'RECORD_LAYOUT'
		key = f"{super().subclass_registry[cat]}.{key}"  # define the key for the axis for future fetch
		rtype = self.__class__.fetch(key)
		if type(rtype) is str:  # if the key is not found in the registry, then it is a scalar
			key = 'Scalar_' + rtype  # construct the new key for the scalar as defined in a2l
			cat = 'RECORD_LAYOUT'
			rtype = self.__class__.fetch(key)
		
		return rtype

	@cached_property
	def address(self):
		return hex(int(self.__dict__['Address']['Value']))[2:]   # transform Ecu address to hex string without '0x'

	@cached_property
	def input(self):
		try:
			key = self.__dict__['InputQuantity']['Value']  # define the key for the axis for future fetch
		except KeyError:
			raise KeyError(f'The key "Conversion" is not found in the calibration description or '
						f'the key "Value" is not found in the "Conversion" dict.')
		cat = 'MEASUREMENT'
		key = f"{super().subclass_registry[cat]}.{key}"
		return self.__class__.fetch(key)

In [ ]:
#| export
class DataConversion(Record):
	"""Data conversion object for calibration; a2l section ["PROJECT"]["MODULE"]["COMPU_METHOD"]]""" 
	__CAT = 'COMPU_METHOD'
	Record.subclass_registry[__CAT] = 'DataConversion'

	def __repr__(self):
		try:
			return f'<{self.__class__.__name__}: {self.Name!r}>'
		except AttributeError:
			return super().__repr__() 

In [ ]:
#| export
class DataLayout(Record):
	"""Data type object for calibration; a2l section ["PROJECT"]["MODULE"]["RECORD_LAYOUT"]""" 
	__CAT = 'RECORD_LAYOUT'
	Record.subclass_registry[__CAT] = 'DataLayout'
	# size: int=Field(default=4, description='size of the data in bytes')

	def __repr__(self):
		try:
			return f'<{self.__class__.__name__}: {self.Name!r}>'
		except AttributeError:
			return super().__repr__() 
	

	@property
	def data_type(self):
		try:
			dtype = self.__dict__['FNC_VALUES']['DataType']['Value']  # define the key for the axis for future fetch
		except KeyError:
			try:
				dtype = self.__dict__['AXIS_PTS_X']['DataType']['Value']  # define the key for the axis for future fetch
			except KeyError:
				raise KeyError(f'The key "DataType" is not found in the RECORD_LAYOUT "FNC_VALUES" or "AXIS_PTS_X" section'
							f'or the key "Value" is not found in the "DataType" dict.')
		return dtype

	@cached_property
	def type_size(self):
		match(self.data_type):
			case 'UBYTE' | 'SBYTE' | 'CHAR':
				return 1
			case 'UWORD' | 'SWORD':
				return 2
			case 'ULONG' | 'SLONG' | 'FLOAT32_IEEE':
				return 4
			case 'A_UINT64' | 'A_INT64' | 'FLOAT64_IEEE':
				return 8
			case _:
				raise ValueError(f'Invalid data type {self.data_type}')

In [ ]:
Record.subclass_registry

{'CHARACTERISTIC': 'Calibration',
 'MEASUREMENT': 'Measurement',
 'AXIS_PTS': 'AxisScale',
 'COMPU_METHOD': 'DataConversion',
 'RECORD_LAYOUT': 'DataLayout'}

In [ ]:
#| export
def load_class_type_a2l_lazy(path: Path, jnode_path: Optional[JsonNodePath]=JsonNodePath('/PROJECT/MODULE[]'))->type(Enum):  # return a class type
	""" Search for the calibration key in the A2L file.
	Descripttion: Load the A2L file as a dictionary.
	
	Create record type (enum class) for the calibration parameter for the given a2l json file

	Args:
		path (str): The path to the A2L file.
		section_key (str): The section key to search for the calibration type.

	Returns:
		dict: The A2L file as a dictionary.
	"""
	record_type_keys = []
	with open(path, "r") as f:
		for object in ijson.items(f, jnode_path.lazy_path):
			# print(list(object.keys()))
			record_type_keys += object.keys()
		# keys = [k for k, v in ijson.kvitems(f, prefix) if type(v) is list]t]
		# record_type_key.append(keys)

	RecordTypes = Enum('RecordType', record_type_keys)
	return RecordTypes

In [ ]:
RecordTypes = load_class_type_a2l_lazy(args.path)
pprint(RecordTypes)
set(RecordTypes.__members__.keys())
Record.record_registry

<enum 'RecordType'>


{'AXIS_PTS',
 'CHARACTERISTIC',
 'COMPU_METHOD',
 'GROUP',
 'IF_DATA',
 'LongIdentifier',
 'MEASUREMENT',
 'MOD_COMMON',
 'MOD_PAR',
 'Name',
 'RECORD_LAYOUT'}

In [ ]:
#| export
def load_records_lazy(path: Path, leaves: list[str], jnode_path: Optional[JsonNodePath]=JsonNodePath('/PROJECT/MODULE[]'))->dict[Record]:
	"""load records from a json file lazily

	Args:
		path (Path): path to the json file
		# use ijson no need for  jnode_paths, though sacrificing a little bit efficiency (list[JsonNodePath]): list of JsonNodePath to the leaves
		leaves (list[str]): list of leaf indices to the records, needs to be unique and in the first item of the a2l json file

	Returns:
		dict[str, Record]: dict of Records and its subclasses, indexed by the leaf indices
	"""
	records = {}
	Record.load_types(path, jnode_path= jnode_path)  # init subclass_registry in  Record
	for leaf in leaves:
		prefix = '' 
		with open(path, "r") as f:
			parser = ijson.parse(f)
			while True:
				prefix, event, value = next(parser)
				if value == leaf and prefix.split('.')[-2] == 'Name':  # find the leaf with the key "Name", is the index of  an a2l record
					break
			else:
				raise ValueError(f'leaf {leaf} not found in {path}')
			prefix = ".".join(prefix.split('.')[:-2])  # remove the last two segments "Name" and "Value", return to the root of  the item
			objects =  ijson.kvitems(parser, prefix)
			rec  = {k:v for k,v in objects}
			rec['Name'] = leaf   # add the calibration key as name back to the record
			
			record_type = set(re.split(r'\.', prefix)).intersection(set(RecordTypes.__members__.keys()))
			assert len(record_type)==1, f'Invalid record type/s {record_type}'
			category = record_type.pop()
			cls_name = Record.subclass_registry.get(category, 'Record')
			cls = globals().get(cls_name, Record)

			if inspect.isclass(cls) and issubclass(cls, Record):
				factory = cls
			else:
				factory = Record
			
			key = f'{cls_name}.{leaf}'
			records[key] = factory(**rec)  # create the record object and add it to the record registry
			
	return records

In [ ]:
# jnode_path = '.'.join(jnode_paths[0].lazy_path.split('.')[:-2])
args.path, args.leaves, args.node_path

('../res/vbu_sample.json',
 ['TQD_trqTrqSetNormal_MAP_v',
  'VBU_L045A_CWP_05_09T_AImode_CM_single',
  'Lookup2D_FLOAT32_IEEE',
  'Lookup2D_X_FLOAT32_IEEE',
  'Scalar_FLOAT32_IEEE',
  'TQD_vVehSpd',
  'TQD_vSgndSpd_MAP_y',
  'TQD_pctAccPedPosFlt',
  'TQD_pctAccPdl_MAP_x'],
 <JsonNodePath [<PROJECT dict>, <MODULE[] list>]>)

In [ ]:
Record.load_records(args.path, args.leaves, args.node_path) #, jnode_path= args.node_path.lazy_path)
Record.subclass_registry
sorted(Record.record_registry)

{'CHARACTERISTIC': 'Calibration',
 'MEASUREMENT': 'Measurement',
 'AXIS_PTS': 'AxisScale',
 'COMPU_METHOD': 'DataConversion',
 'RECORD_LAYOUT': 'DataLayout'}

['AxisScale.TQD_pctAccPdl_MAP_x',
 'AxisScale.TQD_vSgndSpd_MAP_y',
 'Calibration.TQD_trqTrqSetNormal_MAP_v',
 'DataConversion.VBU_L045A_CWP_05_09T_AImode_CM_single',
 'DataLayout.Lookup2D_FLOAT32_IEEE',
 'DataLayout.Lookup2D_X_FLOAT32_IEEE',
 'DataLayout.Scalar_FLOAT32_IEEE',
 'Measurement.TQD_pctAccPedPosFlt',
 'Measurement.TQD_vVehSpd']

In [ ]:
records = load_records_lazy(args.path, args.leaves, args.node_path)

pprint(sorted(records.items()))
# pprint(records)


[('AxisScale.TQD_pctAccPdl_MAP_x', <AxisScale: 'TQD_pctAccPdl_MAP_x'>),
 ('AxisScale.TQD_vSgndSpd_MAP_y', <AxisScale: 'TQD_vSgndSpd_MAP_y'>),
 ('Calibration.TQD_trqTrqSetNormal_MAP_v',
  <Calibration: 'TQD_trqTrqSetNormal_MAP_v'>),
 ('DataConversion.VBU_L045A_CWP_05_09T_AImode_CM_single',
  <DataConversion: 'VBU_L045A_CWP_05_09T_AImode_CM_single'>),
 ('DataLayout.Lookup2D_FLOAT32_IEEE', <DataLayout: 'Lookup2D_FLOAT32_IEEE'>),
 ('DataLayout.Lookup2D_X_FLOAT32_IEEE',
  <DataLayout: 'Lookup2D_X_FLOAT32_IEEE'>),
 ('DataLayout.Scalar_FLOAT32_IEEE', <DataLayout: 'Scalar_FLOAT32_IEEE'>),
 ('Measurement.TQD_pctAccPedPosFlt', <Measurement: 'TQD_pctAccPedPosFlt'>),
 ('Measurement.TQD_vVehSpd', <Measurement: 'TQD_vVehSpd'>)]


In [ ]:
key = 'DataLayout.' + args.leaves[4]
print(key)
record_type = records[key]
record_type, record_type.data_type, record_type.type_size

DataLayout.Scalar_FLOAT32_IEEE


(<DataLayout: 'Scalar_FLOAT32_IEEE'>, 'FLOAT64_IEEE', 8)

In [ ]:

key = 'Calibration.' + args.leaves[0]
print(key)
records[key]

Calibration.TQD_trqTrqSetNormal_MAP_v


<Calibration: 'TQD_trqTrqSetNormal_MAP_v'>

In [ ]:

calib = Record.fetch(key)
pprint(calib)

calib.axes[0].axis_scale.input
calib.axes[0].axis_scale.input.record_type


<Calibration: 'TQD_trqTrqSetNormal_MAP_v'>


<Measurement: 'TQD_pctAccPedPosFlt'>

'Scalar_FLOAT32_IEEE'

In [ ]:
calib.record_type
calib.record_type.data_type
calib.record_type.type_size
calib.axes[0].axis_scale.record_type.type_size
#calib.axes[0].axis_scale.input.record_type.data_type) 
#calib.axes[0].axis_scale.input.record_type.type_size)
calib.axes[0].axis_scale.data_conversion
calib.axes[0].data_conversion.Format

<DataLayout: 'Lookup2D_FLOAT32_IEEE'>

'FLOAT64_IEEE'

8

8

<DataConversion: 'VBU_L045A_CWP_05_09T_AImode_CM_single'>

{'Value': '%8.6'}

In [ ]:

for k,v in records.items():
	pprint(v)

pprint(records['Calibration.TQD_trqTrqSetNormal_MAP_v'].axes[0].measurement.data_conversion.Name)

<Calibration: 'TQD_trqTrqSetNormal_MAP_v'>
<DataConversion: 'VBU_L045A_CWP_05_09T_AImode_CM_single'>
<DataLayout: 'Lookup2D_FLOAT32_IEEE'>
<DataLayout: 'Lookup2D_X_FLOAT32_IEEE'>
<DataLayout: 'Scalar_FLOAT32_IEEE'>
<Measurement: 'TQD_vVehSpd'>
<AxisScale: 'TQD_vSgndSpd_MAP_y'>
<Measurement: 'TQD_pctAccPedPosFlt'>
<AxisScale: 'TQD_pctAccPdl_MAP_x'>
'VBU_L045A_CWP_05_09T_AImode_CM_single'


In [ ]:
'0' < '1'

True

In [ ]:
#| export
class XCPConfig(BaseModel):
	"""XCP configuration for the calibration parameter"""
	channel: int = Field(default=3, ge=0, le=10000, description='XCP channel')
	download_can_id: str = Field(default='630', ge='0', alias='download', validate_default=True, description='CAN ID for download')
	upload_can_id: str = Field(default='631', ge='0', alias='upload', validate_default=True, description='CAN ID for upload')


In [ ]:

config = XCPConfig(channel=3, download='630', upload='631')
config.model_dump()

{'channel': 3, 'download_can_id': '630', 'upload_can_id': '631'}

In [ ]:
#| export
class XCPData(BaseModel):
	"""XCP data for the calibration parameter"""
	name: str = Field(default='TQD_trqTrqSetNormal_MAP_v', description='XCP calibration name')
	address: Optional[str] = Field(default='7000aa2a',pattern=r'^[0-9A-Fa-f]{8}$', description='Target Ecu address')
	dim: conlist(Annotated[int,Field(gt=0,lt=50)],min_length=2,max_length=2)
	value_type: str = Field(default='FLOAT32_IEEE', description='XCP data type')
	value_length: int = Field(default=4,multiple_of=2,gt=0,description='XCP data type length in Bytes')
	value: str = Field(pattern=r'^[0-9A-Fa-f]{0,3000}$',  min_length=1, max_length=3000, description='XCP calbiration data')

In [ ]:
def Get_Init_XCPData(path: Path=Path('../res/init_value.json'))->List[XCPData]:

	xcp_data = []
	with open(Path('../res/init_value.json')) as f:   
		init_values = json.load(f)
		data = init_values['data']
		for v in data:
			xcp_data.append(XCPData(**v))
	
	return xcp_data

In [ ]:
xcp_data = Get_Init_XCPData('../res/init_value.json')
len(xcp_data)
xcp_data[0].name, xcp_data[0].dim, xcp_data[0].value_type, xcp_data[0].value_length, len(xcp_data[0].value)
xcp_data[1].name, xcp_data[1].dim, xcp_data[1].value_type, xcp_data[1].value_length, len(xcp_data[1].value)

2

('TQD_trqTrqSetNormal_MAP_v', [21, 17], 'FLOAT32_IEEE', 4, 2856)

('TQD_trqTrqSetECO_MAP_v', [21, 17], 'FLOAT32_IEEE', 4, 2856)

In [ ]:
#| export
class XCPCalib(BaseModel):
	"""XCP calibration parameter"""
	config: XCPConfig = Field(default_factory=XCPConfig, description='XCP configuration')
	data: List[XCPData] = Field(default_factory=List[XCPData], description='list of XCP calibration data')

In [ ]:
#| export
def Generate_XCPData(
		a2l: Path=Path('../res/vbu_sample.json'), 
		keys: List[str]=['TQD_trqTrqSetNormal_MAP_v',
					"VBU_L045A_CWP_05_09T_AImode_CM_single",
					"Lookup2D_FLOAT32_IEEE",
					"Lookup2D_X_FLOAT32_IEEE, " 
					"TQD_vVehSpd",
					"TQD_vSgndSpd_MAP_y",
					"TQD_pctAccPedPosFlt",
					"TQD_pctAccPdl_MAP_x"],
		node_path: str='/PROJECT/MODULE[]',
		default_xcpdata: str=2856*'0')->XCPCalib:
	"""Generate XCP calibration parameter from A2L file and calibration parameter name

	Args:
		a2l (Path): path to the A2L file
		calib (str): calibration parameter name

	Returns:
		XCPCalib: XCP calibration parameter
	"""

	# load calibration parameter from A2L file
	calibs = load_records_lazy(a2l, keys, JsonNodePath(node_path))
	idx = 'Calibration.' + keys[0]
	calib = calibs[idx]

	# create XCP calibration parameter
	xcp_data = XCPData(name=calib.Name, 
					address=calib.address, 
					dim=[calib.axes[0].MaxAxisPoints['Value'],
						calib.axes[1].MaxAxisPoints['Value']], 
					value_type=calib.record_type.data_type,
					value_length=calib.record_type.type_size,
					value=default_xcpdata)

	return xcp_data

In [ ]:
args.leaves, args.node_path.node_path_str

(['TQD_trqTrqSetNormal_MAP_v',
  'VBU_L045A_CWP_05_09T_AImode_CM_single',
  'Lookup2D_FLOAT32_IEEE',
  'Lookup2D_X_FLOAT32_IEEE',
  'Scalar_FLOAT32_IEEE',
  'TQD_vVehSpd',
  'TQD_vSgndSpd_MAP_y',
  'TQD_pctAccPedPosFlt',
  'TQD_pctAccPdl_MAP_x'],
 '/PROJECT/MODULE[], ')

In [ ]:

init_xcpdata = Get_Init_XCPData('../res/init_value.json')
# xcp_data[0].name, xcp_data[0].dim, xcp_data[0].value_type, xcp_data[0].value_length, len(xcp_data[0].value)
xcp_data = Generate_XCPData(a2l=Path('../res/vbu_sample.json'),
							keys=args.leaves,
							node_path=args.node_path.node_path_str,
							default_xcpdata=init_xcpdata[0].value)

In [ ]:
Record.record_registry

{'Calibration.TQD_trqTrqSetNormal_MAP_v': <Calibration: 'TQD_trqTrqSetNormal_MAP_v'>,
 'DataConversion.VBU_L045A_CWP_05_09T_AImode_CM_single': <DataConversion: 'VBU_L045A_CWP_05_09T_AImode_CM_single'>,
 'DataLayout.Lookup2D_FLOAT32_IEEE': <DataLayout: 'Lookup2D_FLOAT32_IEEE'>,
 'DataLayout.Lookup2D_X_FLOAT32_IEEE': <DataLayout: 'Lookup2D_X_FLOAT32_IEEE'>,
 'DataLayout.Scalar_FLOAT32_IEEE': <DataLayout: 'Scalar_FLOAT32_IEEE'>,
 'Measurement.TQD_vVehSpd': <Measurement: 'TQD_vVehSpd'>,
 'AxisScale.TQD_vSgndSpd_MAP_y': <AxisScale: 'TQD_vSgndSpd_MAP_y'>,
 'Measurement.TQD_pctAccPedPosFlt': <Measurement: 'TQD_pctAccPedPosFlt'>,
 'AxisScale.TQD_pctAccPdl_MAP_x': <AxisScale: 'TQD_pctAccPdl_MAP_x'>}

In [ ]:
key = 'DataLayout.FLOAT32_IEEE'
key.split('.')[-1]

'FLOAT32_IEEE'

In [ ]:
calib = Record.fetch('Calibration.TQD_trqTrqSetNormal_MAP_v')
pprint(calib)

calib.record_type
calib.axes[0].axis_scale.record_type
calib.axes[0].axis_scale.input.record_type
calib.axes[0].axis_scale.data_conversion
calib.axes[0].data_conversion.Format

<Calibration: 'TQD_trqTrqSetNormal_MAP_v'>


<DataLayout: 'Lookup2D_FLOAT32_IEEE'>

<DataLayout: 'Lookup2D_X_FLOAT32_IEEE'>

'Scalar_FLOAT32_IEEE'

<DataConversion: 'VBU_L045A_CWP_05_09T_AImode_CM_single'>

{'Value': '%8.6'}

In [ ]:
pprint(calib.axes[1].axis_scale.record_type)

<DataLayout: 'Lookup2D_X_FLOAT32_IEEE'>


In [ ]:
#| export
def load_a2l_lazy(path: Path, leaves: list[str])->dict:
	""" Search for the calibration key in the A2L file.
	Descripttion: Load the A2L file as a dictionary.

	Args:
		path (str): The path to the A2L file.
		calib_key (str): The node path to the calibration parameters.

	Returns:
		dict: The A2L file as a dictionary.
	"""
	records = []
	for leaf in leaves:
		prefix = ''
		with open(path, "r") as f:
			parser = ijson.parse(f)
			while True:
				prefix, event, value = next(parser)
				if value == leaf:
					break
			else:
				raise ValueError(f'Key {key} not found in the A2L file.')
			
			prefix = ".".join(prefix.split('.')[:-2])  # remove the last two segments "Name" and "Value", return to the root of  the item
			objects =  ijson.kvitems(parser, prefix)
			record  = {k:v for k,v in objects}
			record['Name'] = leaf   # add the calibration key as name back to the record
			records.append(record)

	return records

In [ ]:
# parser = get_argparser()
args = parser.parse_args(
	[
		"-p",
		# r"../res/VBU_AI.json",
		r"../res/VBU_AI.json",
		"-n",
		r"/PROJECT/MODULE[], ",
		# r"/PROJECT/MODULE[]/CHARACTERISTIC[], "
		# 	r"/PROJECT/MODULE[]/MEASUREMENT[], "
		# 	r"/PROJECT/MODULE[]/AXIS_PTS[], "
		# 	r"/PROJECT/MODULE[]/COMPU_METHOD[]",
		"-l",
		r"TQD_trqTrqSetNormal_MAP_v, " 
				r"VBU_L045A_CWP_05_09T_AImode_CM_single, " 
				r"Lookup2D_FLOAT32_IEEE, "
				r"Lookup2D_X_FLOAT32_IEEE, "
				r"TQD_vVehSpd, "
				r"TQD_vSgndSpd_MAP_y, "
				r"TQD_pctAccPedPosFlt, "
				r"TQD_pctAccPdl_MAP_x",
	]
)
# args.__dict__
args.path, args.leaves, args.node_path

('../res/VBU_AI.json',
 ['TQD_trqTrqSetNormal_MAP_v',
  'VBU_L045A_CWP_05_09T_AImode_CM_single',
  'Lookup2D_FLOAT32_IEEE',
  'Lookup2D_X_FLOAT32_IEEE',
  'TQD_vVehSpd',
  'TQD_vSgndSpd_MAP_y',
  'TQD_pctAccPedPosFlt',
  'TQD_pctAccPdl_MAP_x'],
 <JsonNodePath [<PROJECT dict>, <MODULE[] list>]>)

In [ ]:
# %%timeit -n 3 -r 4
records = load_a2l_lazy(args.path, args.leaves)

In [ ]:
len(records)
records[0]
# pprint(records)

8

{'LongIdentifier': {},
 'Type': 'VALUE',
 'Address': {'Value': '1879110963', 'Base': 16, 'Size': 8},
 'Deposit': {'Value': 'Scalar_UBYTE'},
 'MaxDiff': {},
 'Conversion': {'Value': 'VBU_L045A_CWP_05_09T_AImode_CM_uint8'},
 'LowerLimit': {},
 'UpperLimit': {'Value': 255},
 'AXIS_DESCR': [{'Attribute': 'COM_AXIS',
   'InputQuantity': {'Value': 'NO_INPUT_QUANTITY'},
   'Conversion': {'Value': 'VBU_L045A_CWP_05_09T_AImode_CM_single'},
   'MaxAxisPoints': {'Value': 8, 'Base': 10, 'Size': 1},
   'LowerLimit': {'Value': Decimal('-3.4E+38'),
    'IntegralSign': '-',
    'IntegralSize': 1,
    'DecimalSize': 5},
   'UpperLimit': {'Value': Decimal('3.4E+38'),
    'IntegralSize': 1,
    'DecimalSize': 5},
   'AXIS_PTS_REF': {'AxisPoints': {'Value': 'CaSOF_pctBuffUseFacSocTemp_MAP_y'}}},
  {'Attribute': 'COM_AXIS',
   'InputQuantity': {'Value': 'NO_INPUT_QUANTITY'},
   'Conversion': {'Value': 'VBU_L045A_CWP_05_09T_AImode_CM_single'},
   'MaxAxisPoints': {'Value': 5, 'Base': 10, 'Size': 1},
   'Low

In [ ]:
#| export
def load_a2l_eager(path: Path, jnode_path: JsonNodePath=JsonNodePath('/PROJECT/MODULE[]'))->dict:
	""" Load the A2L file as a dictionary.
	Descripttion: Load the A2L file as a dictionary.

	Args:
		path (Path): The path to the A2L file.
		node (str): The node to search for, e.g. "/PROJECT/MODULE[0]/CHARACTERISTIC".

	Returns:
		dict: The A2L file as a dictionary.
	"""
	records = {}
	path_list = re.split(r'\.', jnode_path.lazy_path)[:-1]
	with open(path, "r") as f:
		n = json.load(f)
		for p in path_list:
			n = n[p] 
		# only the first module is used
		# sections = ['CHARACTERISTIC', 'MEASUREMENT', 'AXIS_PTS', 'COMPU_METHOD']
		# for s in sections:
		# 	for key, value in n[s].items():
		# 		print(key, value)
				# if prefix.endswith(".ECU_ADDRESS"):
				# 	yield value
			# a2l = json.load(f)
	return n

In [ ]:
args.path, args.node_path

('../res/VBU_AI.json', <JsonNodePath [<PROJECT dict>, <MODULE[] list>]>)

In [ ]:
# %%timeit -n 3 -r 4
# calibs = load_a2l_eager(args.path, args.node_path)

In [ ]:
# calibs[0]['CHARACTERISTIC'][0]['Name']
# pprint(calibs)

In [ ]:
# %%timeit -r 4 -n 10000
# f = open(args.path, "r")
f = open("../res/vbu_sample.json", "r")
raw_data = json.load(f)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()